# Task 1. Import relevant libraries and the ords_prods_merge dataframe

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

# Defining path to the data folder (main directory)

In [2]:
path = r'C:\Users\New User\OneDrive\Documents\05-2024 Instacart Basket Analysis\02 Data\Prepared Data'

# Load the dataframe

In [3]:
ords_prods_merge = pd.read_pickle(f'{path}/orders_products_combined.pkl')

#  Display the first few rows of the dataframe to confirm successful import

In [4]:
ords_prods_merge.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,_merge,product_name,aisle_id,department_id,prices,merge_flag,busiest_day,busiest_days,busiest_period_of_day
0,2,33120.0,1.0,1.0,202279,3,5,9,8.0,both,Organic Egg Whites,86.0,16.0,11.3,both,Regularly busy,Regular days,Most orders
1,2,28985.0,2.0,1.0,202279,3,5,9,8.0,both,Michigan Organic Kale,83.0,4.0,13.4,both,Regularly busy,Regular days,Most orders
2,2,9327.0,3.0,0.0,202279,3,5,9,8.0,both,Garlic Powder,104.0,13.0,3.6,both,Regularly busy,Regular days,Most orders
3,2,45918.0,4.0,1.0,202279,3,5,9,8.0,both,Coconut Butter,19.0,13.0,8.4,both,Regularly busy,Regular days,Most orders
4,2,30035.0,5.0,0.0,202279,3,5,9,8.0,both,Natural Sweetener,17.0,13.0,13.7,both,Regularly busy,Regular days,Most orders


# Task 2. Find the aggregated mean of the “order_number” column grouped by “department_id” for the entire dataframe

In [5]:
# Group by department_id and calculate the mean of order_number

dept_order_mean = ords_prods_merge.groupby('department_id')['order_number'].mean()

In [6]:
# Check output

(dept_order_mean)

department_id
1.0     15.457838
2.0     17.277920
3.0     17.170395
4.0     17.811403
5.0     15.215751
6.0     16.439806
7.0     17.225802
8.0     15.340650
9.0     15.895474
10.0    20.197148
11.0    16.170638
12.0    15.887671
13.0    16.583536
14.0    16.773669
15.0    16.165037
16.0    17.665606
17.0    15.694469
18.0    19.310397
19.0    17.177343
20.0    16.473447
21.0    22.902379
Name: order_number, dtype: float64

# Task 3. Analyze the results

In [8]:
# Analysis of Aggregated Means

# The results show the mean number of orders per department. 

# By comparing these means, we can identify which departments have the highest and lowest average order counts. 

# This insight can help in understanding customer purchasing patterns across different departments.

# Task 4. Create a loyalty flag for existing customers

# Create the max_order column

In [9]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform('max')

# Create the loyalty_flag column

In [10]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [11]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] > 10) & (ords_prods_merge['max_order'] <= 40), 'loyalty_flag'] = 'Regular customer'

In [12]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

# Verify the creation of the loyalty flag

In [13]:
ords_prods_merge[['user_id', 'max_order', 'loyalty_flag']].head()

,user_id,max_order,loyalty_flag
0,202279,9,New customer
1,202279,9,New customer
2,202279,9,New customer
3,202279,9,New customer
4,202279,9,New customer


# Display the value counts for the new 'loyalty_flag' column

In [14]:
ords_prods_merge['loyalty_flag'].value_counts()

loyalty_flag
Regular customer    16381472
Loyal customer      10679578
New customer         5580218
Name: count, dtype: int64

# Task 5. Check the basic statistics of product prices for each loyalty category

# Calculate basic statistics

In [15]:
loyalty_price_stats = ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

In [17]:
loyalty_price_stats

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal customer,10651404.0,10.377144,329.068187,1.0,4.2,7.4,11.2,99999.0
New customer,5470817.0,13.208437,589.963575,1.0,4.2,7.4,11.3,99999.0
Regular customer,16282638.0,12.616225,548.767475,1.0,4.2,7.4,11.3,99999.0


# Task 6. Create a spending flag based on the average price of products purchased by each user

# Step 1. Calculate the mean price for each user

In [18]:
ords_prods_merge['mean_price'] = ords_prods_merge.groupby('user_id')['prices'].transform('mean')

# Step 2. Create the spending_flag column

In [19]:
ords_prods_merge.loc[ords_prods_merge['mean_price'] < 10, 'spending_flag'] = 'Low spender'

In [20]:
ords_prods_merge.loc[ords_prods_merge['mean_price'] >= 10, 'spending_flag'] = 'High spender'

# Verify the creation of the spending flag

In [23]:
ords_prods_merge[['user_id', 'mean_price', 'spending_flag']].head()

,user_id,mean_price,spending_flag
0,202279,8.618889,Low spender
1,202279,8.618889,Low spender
2,202279,8.618889,Low spender
3,202279,8.618889,Low spender
4,202279,8.618889,Low spender


# Task 7. Create an order frequency flag

# Step 1. Calculate the median of days_since_prior_order for each user

In [25]:
ords_prods_merge['median_days_since_last_order'] = ords_prods_merge.groupby('user_id')['days_since_last_order'].transform('median')

# Step 2. Create the order_frequency_flag column

In [27]:
ords_prods_merge.loc[ords_prods_merge['median_days_since_last_order'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [28]:
ords_prods_merge.loc[(ords_prods_merge['median_days_since_last_order'] > 10) & (ords_prods_merge['median_days_since_last_order'] <= 20), 'order_frequency_flag'] = 'Regular customer'

In [29]:
ords_prods_merge.loc[ords_prods_merge['median_days_since_last_order'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

# Verify the creation of the order frequency flag

In [31]:
ords_prods_merge[['user_id', 'median_days_since_last_order', 'order_frequency_flag']].head()

,user_id,median_days_since_last_order,order_frequency_flag
0,202279,24.0,Non-frequent customer
1,202279,24.0,Non-frequent customer
2,202279,24.0,Non-frequent customer
3,202279,24.0,Non-frequent customer
4,202279,24.0,Non-frequent customer


# Task 9. Export your dataframe as a pickle file

In [32]:
ords_prods_merge.to_pickle(f'{path}/orders_products_merged_updated.pkl')

# Verify if the exported dataframe was successful or not 

In [38]:
print("Dataframe exported successfully to:", export_path)

Dataframe exported successfully to: C:\Users\New User\OneDrive\Documents\05-2024 Instacart Basket Analysis\02 Data\Prepared Data\02 Data\Prepared Data\orders_products_merged_updated.pkl
